Imports

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
pip install scipy 
# Used for the checksum in the dataset download.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision.datasets import Flowers102
from torch.utils.data import Subset, ConcatDataset
from collections import defaultdict, Counter


Checking what device is available

In [ ]:
print("Torch:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Constants

In [36]:
DATASET_ROOT: str = "./data" 
TRAIN_RELATIVE_SIZE: float = 0.5
VALIDATION_RELATIVE_SIZE: float = 0.25
TEST_RELATIVE_SIZE: float = 0.25
YOLOV5_MODEL = 'yolov5s'

Fetching the database

In [ ]:
# Load the entire dataset to be processed later.
full_dataset = torch.utils.data.ConcatDataset([
    Flowers102(root=DATASET_ROOT, split="train", download=True),
    Flowers102(root=DATASET_ROOT, split="val",   download=True),
    Flowers102(root=DATASET_ROOT, split="test",  download=True),
]) 

Database information

In [ ]:
labels = np.asarray(full_dataset._labels, dtype=int)
counts = Counter(labels)

num_classes = len(counts)
total = len(labels)
min_c = min(counts.values())
max_c = max(counts.values())

print(f"Total samples: {total}")
print(f"Classes: {num_classes}")
print(f"Min per class: {min_c}")
print(f"Max per class: {max_c}")
print(f"Imbalance ratio (max/min): {max_c/min_c:.2f}")

Total samples: 8189
Classes: 102
Min per class: 40
Max per class: 258
Imbalance ratio (max/min): 6.45


Data Preparation

Added utility functions to properly separate data.

In [24]:
def extract_all_labels_from_concat_dataset(concat_dataset: ConcatDataset):
    """Collect labels from each underlying dataset without loading images."""
    all_labels_list = []

    for single_dataset in concat_dataset.datasets:
        if hasattr(single_dataset, "_labels"):          # Flowers102
            labels_array = np.asarray(single_dataset._labels, dtype=int)
        elif hasattr(single_dataset, "targets"):        # ImageFolder, CIFAR, etc.
            labels_array = np.asarray(single_dataset.targets, dtype=int)
        else:  # fallback (slow)
            labels_array = np.array(
                [single_dataset[i][1] for i in range(len(single_dataset))],
                dtype=int
            )

        all_labels_list.append(labels_array)

    return np.concatenate(all_labels_list)


def stratified_split_concat_dataset(
    concat_dataset: ConcatDataset,
    split_fractions=(0.8, 0.1, 0.1),
    random_seed=42
):
    """Split ConcatDataset into stratified Subsets with same class proportions."""

    normalized_fractions = np.array(split_fractions, dtype=float)
    normalized_fractions = normalized_fractions / normalized_fractions.sum()

    all_labels = extract_all_labels_from_concat_dataset(concat_dataset)
    random_generator = np.random.default_rng(random_seed)

    # group global indices by class
    class_to_indices = defaultdict(list)
    for global_index, class_label in enumerate(all_labels):
        class_to_indices[int(class_label)].append(global_index)

    split_indices_per_subset = [[] for _ in range(len(normalized_fractions))]

    for class_indices in class_to_indices.values():
        class_indices = np.array(class_indices, dtype=int)
        random_generator.shuffle(class_indices)

        total_class_samples = len(class_indices)
        samples_per_split = np.floor(normalized_fractions * total_class_samples).astype(int)

        # distribute leftover samples
        remainder = total_class_samples - samples_per_split.sum()
        for split_id in random_generator.permutation(len(samples_per_split))[:remainder]:
            samples_per_split[split_id] += 1

        start_pointer = 0
        for split_id, count in enumerate(samples_per_split):
            split_indices_per_subset[split_id].extend(
                class_indices[start_pointer:start_pointer + count].tolist()
            )
            start_pointer += count

    # shuffle each split's indices
    for split_list in split_indices_per_subset:
        random_generator.shuffle(split_list)

    train_subset, val_subset, test_subset = [
        Subset(concat_dataset, indices) for indices in split_indices_per_subset
    ]

    return train_subset, val_subset, test_subset

Training YoloV5

In [ ]:
model = torch.hub.load("ultralytics/yolov5", YOLOV5_MODEL, pretrained=True)
model = model.to(device)
model.train()

In [35]:
print("REMEMBER TO DO MULTIPLE SPLITS HONEY")
train_dataset, val_dataset, test_dataset = stratified_split_concat_dataset(
    full_dataset,
    split_fractions=(TRAIN_RELATIVE_SIZE, VALIDATION_RELATIVE_SIZE, TEST_RELATIVE_SIZE),
    random_seed=42
)
train_dataset[0]

REMEMBER TO DO MULTIPLE SPLITS HONEY


(<PIL.Image.Image image mode=RGB size=666x500>, 19)

Training VGG

Evaluating Models

Model Evaluation Functions

In [33]:
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')
print('Model evaluation functions')

Model evaluation functions
Model evaluation functions
Model evaluation functions
Model evaluation functions
Model evaluation functions
Model evaluation functions
Model evaluation functions
Model evaluation functions
Model evaluation functions
Model evaluation functions
Model evaluation functions
